# Experiment Results
## Experiment picker

Use the widget below to select and load experiments from `data/`.

**Mode Selection:**
- **Single Experiment** (default): Select one experiment and automatically load all its configurations. Uses a radio button for experiment selection.
- **Multiple Experiments**: Select multiple experiments and customize which configurations to load for each. Uses checkboxes for selection.

**Configuration Handling:**
- If an experiment contains configuration subfolders (for example `data/exp/<cfg>/001`), it will be treated as a multi-config experiment and config pickers will be shown.
- If the experiment has `001` directly under `data/exp/001`, it will be treated as a single-run experiment and no config picker will be shown.

After selecting experiments and configurations, click **Choose experiment(s)** to load the data. The results will be printed and saved to `picker.last_loaded`, and a CSV picker will automatically appear to select which data files to load.

> Tip: If the widgets do not show or importing `plothelpers2` raises an error, install the widget dependency with:
> ```bash
> pip install ipywidgets
> ```


In [ ]:
# EXP picker
from pathlib import Path
import plothelpers2 as ph

# create and show the picker (data folder is relative to this notebook)
DATA_DIR = Path('data')
picker = ph.create_and_show_picker(DATA_DIR)

# Wrap the original load button callback to also show CSV picker after loading
original_callback = picker.load_button._click_handlers.callbacks[0] if picker.load_button._click_handlers.callbacks else None

def enhanced_load_callback(btn):
    # Call original callback
    if original_callback:
        original_callback(btn)
    
    # After experiments are loaded, automatically show CSV picker
    if picker.last_loaded:
        get_selections = ph.create_csv_picker_for_loaded_paths(picker, DATA_DIR)
        globals()['get_selections'] = get_selections

# Replace the callback
picker.load_button._click_handlers.callbacks = []
picker.load_button.on_click(enhanced_load_callback)

## Data Preview

In [ ]:
# Preview helper: choose experiment & see a combined preview (no csv_basename key anymore)
ph.show_loaded_preview()


## Time elapsed between blocks (histogram)

In [ ]:
# TELAPSED histogram
ph.show_telapsed_histogram()


## Time elapsed between block production to reception

In [ ]:
# TPROD histogram - time between block productions
ph.show_tprod_histogram()


## Block propegation time

In [ ]:
# Block propagation time boxplot
ph.show_block_propagation_boxplot()
